In [ ]:
# import all libraries

import os
import sys
import mne
import fnmatch
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('C:/Users/Heinz Lab Analysis/Downloads/ANLffr-master/') # change folder destination for the anlffr directory
from anlffr.helper import biosemi2mne as bs
from anlffr.preproc import find_blinks
from mne import compute_proj_epochs

In [ ]:
# loading just one file 
x1='//datadepot.rcac.purdue.edu/depot/heinz/data/UserTESTS/MP/Chin EEG data/bdf data/042925/Q513_STM_tone_nonoise.bdf'# name and address of the file you want to load
raw,eves=bs.importbdf(x1,verbose='DEBUG',refchans=['EXG1', 'EXG2'])# raw and eves extracted from the selected file


In [ ]:
# loading multiple files with same name
fl_dir='//datadepot.rcac.purdue.edu/depot/heinz/data/UserTESTS/MP/Chin EEG data/bdf data/042925/' # name of the folder where your file is present
os.chdir(fl_dir) # changing directory to data folder
fl_nm = 'Q513_STM_tone_nonoise' # name of the file you want to load
bdfs = fnmatch.filter(os.listdir(),fl_nm + '*.bdf') # string matching to find all the files which has the string of the file name
bdfs.sort(key=os.path.getmtime)# sorting them according to order in which they were collected

# for loop to load each of the bdfs files one by one and concatenate the raw and eves structure
rawlist=[]
eveslist=[]

for bdf in bdfs:
    rawtemp,evestemp=bs.importbdf(fl_dir+'/'+bdf,verbose='DEBUG',refchans=['EXG1', 'EXG2'])
    rawlist+=[rawtemp, ]
    eveslist+=[evestemp, ]
raw, eves = mne.concatenate_raws(rawlist, events_list=eveslist)


In [ ]:
# setting headcap montage - in case if you want to make topological maps. You can skip this if you dont get any error in the topo map step
montage = mne.channels.make_standard_montage("biosemi32")
raw.set_eeg_reference(projection=True)
mdc = dict(zip(raw.ch_names,montage.ch_names))
raw.rename_channels(mdc)
raw.set_montage(montage,on_missing='ignore')
raw.pick_channels(montage.ch_names)


In [ ]:
# filter - low band pass for scalp and high band pass for subdermal
raw1=raw
raw=raw1
raw.filter(1.,40.) # for scalp  # leave some frequencies on the left hand side to get rid of the DC component
raw.filter(40.,4000.) # for subdermal

In [ ]:
# visualise the raw plots to find bad channels and identify noisy channels - you may choose not to do it
%matplotlib
raw.plot(duration=25.0, n_channels=32, scalings=dict(eeg=200e-6))

In [ ]:
# remove bad channels that you think is noise
# just some examples
raw.info['bads'].append('A12') 
raw.info['bads'].append('A26') 
raw.info['bads'].append('A27') 
raw.info['bads'].append('A20') 

In [ ]:
# blink rejection - only for humans
blinks = find_blinks(raw)
raw.plot(events=blinks, duration=25.0, n_channels=32, scalings=dict(eeg=200e-6)) # visualize blink events
epochs_blinks = mne.Epochs(raw, blinks, event_id=998, baseline=(-0.25, 0.25),reject=dict(eeg=500e-6), tmin=-0.25, tmax=0.25)
blink_proj = compute_proj_epochs(epochs_blinks, n_eeg=1)
raw.add_proj(blink_proj)

raw.plot(duration=25.0, n_channels=32, scalings=dict(eeg=200e-6)) # visualize data after blink rejection

In [ ]:
# epoch the data 
epochs_1= mne.Epochs(raw, eves, event_id=1, baseline=(-0.3, 0), proj=True,tmin=-0.3, tmax=2.2, reject=dict(eeg=200e-6))
epochs_2= mne.Epochs(raw, eves, event_id=2, baseline=(-0.3, 0), proj=True,tmin=-0.3, tmax=2.2, reject=dict(eeg=200e-6))
t_full=epochs_1.times
picks = (6, 7, 8,21, 22, 23, 28, 29, 13) # select channels that you want to work with

ep1_all=(epochs_1.get_data()[:,picks,:]).mean(axis=1)
ep2_all=(epochs_2.get_data()[:,picks,:]).mean(axis=1)

t_all=epochs_1.times

ep1_mean=ep1_all.mean(axis=0)
ep1_sem=ep1_all.std(axis=0)/np.sqrt(ep1_all.shape[0])
ep2_mean=ep2_all.mean(axis=0)
ep2_sem=ep2_all.std(axis=0)/np.sqrt(ep2_all.shape[0])

In [ ]:
# plot mean epoched responses for all trigger types 

plt.figure(figsize=(15, 5))
plt.subplot(1,2,1)
plt.plot(t_full, ep1_mean,label='trig-1',color='blue')
plt.fill_between(t_full, ep1_mean-ep1_sem, ep1_mean+ep1_sem,alpha=0.3,color='blue')

plt.subplot(1,2,2)
plt.plot(t_full, ep2_mean,label='trig-2',color='red')
plt.fill_between(t_full, ep2_mean-ep2_sem, ep2_mean+ep2_sem,alpha=0.3,color='red')


In [ ]:
# topological spread - only for humans
evoked_1=epochs_1.average() # take average across all iterations
evoked_1.plot_topomap(1.25,size=2) # topo map at time 1.25s

In [ ]:
## for subdermal electrodes
from scipy.signal import detrend

raw=raw1
raw.filter(40.,4000.)
epochs_1= mne.Epochs(raw, eves, event_id=1, baseline=(-0.3, 0), proj=True,tmin=-0.3, tmax=2.2, reject=dict(eeg=200e-6),detrend=1)
epochs_2= mne.Epochs(raw, eves, event_id=2, baseline=(-0.3, 0), proj=True,tmin=-0.3, tmax=2.2, reject=dict(eeg=200e-6),detrend=1)

ep1_sub=(epochs_1.get_data()[:,35,:]-epochs_1.get_data()[:,36,:])
ep2_sub=(epochs_2.get_data()[:,35,:]-epochs_2.get_data()[:,36,:])

ep1_sub=(detrend(ep1_sub,axis=1))
ep2_sub=(detrend(ep2_sub,axis=1))

ep1_mean=ep1_sub.mean(axis=0)
ep1_sem=ep1_sub.std(axis=0)/np.sqrt(ep1_sub.shape[0])
ep2_mean=ep2_sub.mean(axis=0)
ep2_sem=ep2_sub.std(axis=0)/np.sqrt(ep2_sub.shape[0])


plt.figure(figsize=(15, 5))
plt.subplot(1,2,1)
plt.plot(t_full, ep1_mean,label='trig-1',color='blue')
plt.fill_between(t_full, ep1_mean-ep1_sem, ep1_mean+ep1_sem,alpha=0.3,color='blue')

plt.subplot(1,2,2)
plt.plot(t_full, ep2_mean,label='trig-2',color='red')
plt.fill_between(t_full, ep2_mean-ep2_sem, ep2_mean+ep2_sem,alpha=0.3,color='red')
